In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [2]:
class Boltznann_two_spins:
    """Class Boltzmann computes an optimal wave function for
    a given hamiltonian and initial w = a, b, W """
    def __init__(self, hamiltonian, w):
        self.hamiltonian = hamiltonian    # function which is defined outside
        self.w = w
        self.a, self.b, self.W = w[0], w[1], w[2]
        self.psi = self.compute_psi(self.w)
        self.w_optimal = np.nan
        self.psi_optimal = np.nan
        self.min_energy = np.nan
    
    ########### PSI #################################
    
    def psi_M(self, S, w):
        """Returns a component of a wavefunction
        in the S direction"""
        a, b, W = w[0], w[1], w[2]
        psi = np.exp(np.sum(np.dot(a, S)))
        for i in range(len(W)):
            Fi = 2 * np.cosh(b[i] + np.sum(np.dot(W[i], S)))
            psi *= Fi
        return psi
    
    def compute_psi(self, w):
        """Computes psi for w = a,b,W """
        N = len(w[0])
        psi = np.zeros((2,2))
        psi[0, 0] = self.psi_M((0, 0), w)
        psi[0, 1] = self.psi_M((0, 1), w)
        psi[1, 0] = self.psi_M((1, 0), w)
        psi[1, 1] = self.psi_M((1, 1), w)
        return psi
    
    def find_len_a(self, N):
        """Returns the size of a for a given length 
        N of w = a,b,W in a form of 1D array"""
        l = 0
        while 2 * l + l ** l != N:
            l += 1
        return l
    
    def convert_w(self, w):
        """From w 1D array returns w = a, b, W"""
        l = self.find_len_a(len(w))
        a, b = w[:l], w[l : 2 * l]
        W = np.resize(w[2 * l + 1:], (l, l))
        return a, b, W

    def compute_psi_1(self, w):
        """Computes psi for w 1D array"""
        w1 = self.convert_w(w)
        return self.compute_psi(w1)
    
    def show_psi(self, psi):
        """psi is an 2x2 array for now
        in general it will be N^N array"""
        s = 'psi = '
        for i in range(len(psi)):
            for j in range(len(psi[i])):
                s += str('%.5f' % psi[i,j]) + '|' + str(i) + str(j) + '>' + ' + '
        print(s[:-3])
    
    def normalize(self, psi):
        """Returns normalized psi"""
        return psi / np.sqrt(np.sum(psi ** 2))
    
    def show_psi_optimal(self):
        """Shows optimal psi"""
        if self.psi_optimal is np.nan:
            raise ValueError('The optimal psi is not computed yet')
        self.show_psi(self.psi_optimal)
        
    def show_omega_optimal(self):
        """Shows optimal w"""
        w = self.w_optimal
        a, b, W = self.convert_w(w)
        print('a = ', a, '\nb = ', b, '\nW = ', W)
        
    ########### PSI #################################
    ########### ENERGY ##############################
    
    def avg_energy(self, psi, ham):
        """Returns the average energy for a given  wavefunciton
        and a given hamiltonian
        <E> = <psi|H|psi> / <psi|psi>"""
        psi_star = np.conjugate(psi)
        return np.sum(np.dot(psi_star, hamiltonian(psi))) / np.sum(np.dot(psi_star, psi))

    def avg_energy1(self, w):
        """Returns the average energy for a given configuration w,
        made to work for scipy.optimize.minimize"""
        psi = self.compute_psi_1(w)
        return self.avg_energy(psi, hamiltonian)
    
    def show_min_energy(self):
        if self.min_energy is np.nan:
            raise ValueError('The optimal energy is not computed yet')
        s = 'E_min = ' + str(self.min_energy)
        print(s)
    
    ########### ENERGY ##############################
    ########### OPTIMIZATION ########################
    
    def find_optimal_psi(self, ham, w):
        """For a given hamiltonian searches for the
        ground state, i.e. the psi which minimizes the energy"""
        a, b, W = w[0], w[1], w[2]
    #     x = np.array(a, b, W)
        x0 = np.zeros(len(a) + len(b) + len(W) * len(W[0]))
        for i in range(len(a)):
            x0[i] = a[i]
        for i in range(len(b)):
            x0[len(a) + i] = b[i]
        for i in range(len(W)):
            for j in range(len(W[0])):
                x0[len(a) + len(b) + i * len(W) + j] = W[i][j]
        w_min = minimize(self.avg_energy1, x0)
        self.w_optimal = w_min.x
        self.psi_optimal = self.normalize(self.compute_psi_1(self.w_optimal))
        self.min_energy = self.avg_energy1(self.w_optimal)
    
    ########### OPTIMIZATION ########################

########### HAMILTONIAN #########################

def sigma_x(psi):
    """Applies sum sigma_x_i on a given 
    wavefunction psi and returns the 
    new wavefunciton"""
    psi1 = np.zeros((len(psi), len(psi)))
    for i in range(len(psi)):
        for j in range(len(psi[i])):
            psi1[(i + 1) % 2, j] += psi[i, j]
            psi1[i, (j + 1) % 2] += psi[i, j]
    return psi1

def sigma_z(psi):
    """Applies sum sigma_z_i sigma_z_{i+1}
    on a given wavefunction psi and returns the 
    new wavefunciton"""
    psi1 = np.zeros((len(psi), len(psi)))
    for i in range(len(psi)):
        for j in range(len(psi[i])):
            psi1[i, j] = (-1) ** (i + j) * psi[i, j]
    return psi1

def hamiltonian(psi):
    """Retuns wave function after the TFI Hamiltonian 
    acts on it, with h = 1 (to be modified later)"""
    return h * sigma_x(psi) + sigma_z(psi)

def hamiltonian1(w):
    """Same as hamiltonian, but instead of psi
    we are given w as a 1D array"""
    psi = compute_psi_1(w)
    return hamiltonian(psi)

########### HAMILTONIAN #########################

In [13]:
########### INITIALIZATION ########################

a = 2,1
b = 2,3
W = np.random.rand(2,2)
w = a, b, W
h = 1
boltzmann = Boltznann_two_spins(hamiltonian, w)

########### INITIALIZATION ########################
########### OPTIMZATION ###########################

boltzmann.find_optimal_psi(hamiltonian1, w)

########### OPTIMZATION ###########################
########### EXTRACTING PARAMETERS #################

w_optimal = boltzmann.w_optimal
psi_optimal = boltzmann.psi_optimal
min_energy = boltzmann.min_energy

########### EXTRACTING PARAMETERS #################

In [14]:
boltzmann.show_psi_optimal()
boltzmann.show_min_energy()
boltzmann.show_omega_optimal()
# print('E_min = ', min_energy)

psi = 0.00001|00> + 0.00001|01> + 0.38269|10> + 0.92388|11>
E_min = 1.828433310308617
a =  [ 5.39664002 -0.304902  ] 
b =  [1.95951426 3.0119153 ] 
W =  [[ 1.95795899 -0.77314899]
 [ 3.90283593  1.95795899]]
